In [1]:
import os
print(os.getcwd())  # Prints the current working directory

C:\Users\garin\Documents\GitHub\ParkinsonsDisease


In [4]:
import pandas as pd

In [4]:
import os
import json
# Specify the folder containing the JSON files
folder_path = r"C:\Users\garin\Documents\GitHub\ParkinsonsDisease\pads-parkinsons-disease\patients"
# Initialize a list to store the extracted data
data_list = []
# Loop through each file in the folder and extract data
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r') as file:
            data = json.load(file)
            data_list.append(data)
# Now, `data_list` contains data from all the JSON files
print(data_list)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\garin\\Documents\\GitHub\\ParkinsonsDisease\\pads-parkinsons-disease\\patients'

In [43]:
import csv
# Write the extracted data to a CSV file
csv_file_path = r'C:\Users\subha\OneDrive\Desktop\Datathon\Feb\pads-parkinsons-disease\patients.csv'
with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Write the header (assuming all JSON files have the same structure)
    header = data_list[0].keys()
    writer.writerow(header)
    # Write the data
    for data in data_list:
        writer.writerow(data.values())

In [9]:
# Folder containing JSON files
folder_path1 = r'C:\Users\garin\OneDrive\Numpy 2025\Datathon feb2025\pads-parkinsons-disease-smartwatch-dataset\pads-parkinsons-disease-smartwatch-dataset'
questionnaire_csv = "questionnaire.csv"

# List to store extracted data
data_list1 = []

# Iterate through all JSON files in the folder
for file_name in os.listdir(folder_path1):
    if file_name.endswith(".json"):  # Process only JSON files
        file_path1 = os.path.join(folder_path1, file_name)

        # Open and read JSON file
        with open(file_path1, "r", encoding="utf-8") as file:
            data1 = json.load(file)

            # Extract metadata
            subject_id = data1.get("subject_id", "")
            study_id = data1.get("study_id", "")
            questionnaire_name = data1.get("questionnaire_name", "")

            # Extract each item (question & response)
            for item in data1.get("item", []):
                row = {
                    "subject_id": subject_id,
                    "study_id": study_id,
                    "questionnaire_name": questionnaire_name,
                    "link_id": item.get("link_id", ""),
                    "question_text": item.get("text", ""),
                    "answer": item.get("answer", "")
                }
                data_list1.append(row)

# Convert list to DataFrame
df = pd.DataFrame(data_list1)

# Save DataFrame to CSV
df.to_csv(questionnaire_csv, index=False, encoding="utf-8")

print(f"CSV file saved as {questionnaire_csv}")

CSV file saved as questionnaire.csv


In [1]:
import os
import csv
import json

In [12]:
# Define paths
json_folder = r'C:\Users\garin\OneDrive\Numpy 2025\Datathon feb2025\pads-parkinsons-disease-smartwatch-dataset\pads-parkinsons-disease-smartwatch-dataset\movement'
txt_folder = r'C:\Users\garin\OneDrive\Numpy 2025\Datathon feb2025\pads-parkinsons-disease-smartwatch-dataset\pads-parkinsons-disease-smartwatch-dataset'
output_csv = r'C:\Users\garin\OneDrive\Numpy 2025\Datathon feb2025\pads-parkinsons-disease-smartwatch-dataset\pads-parkinsons-disease-smartwatch-dataset'

# List to store all data
all_data = []

# Loop through each JSON file in the folder
for json_file in os.listdir(json_folder):
    if json_file.endswith(".json"):
        json_path = os.path.join(json_folder, json_file)
        # Load JSON data
        with open(json_path, "r", encoding="utf-8") as f:
            observation = json.load(f)
        subject_id = observation.get("subject_id", "Unknown")
        study_id = observation.get("study_id", "Unknown")
        device_id = observation.get("device_id", "Unknown")
        # Process each session
        for session in observation["session"]:
            record_name = session["record_name"]
            rows = session["rows"]
            for record in session["records"]:
                device_location = record["device_location"]
                channels = record["channels"]
                file_name = record["file_name"]
                # Build full path to .txt file
                txt_path = os.path.join(txt_folder, file_name)
                if os.path.exists(txt_path):  # Check if file exists
                    # Read .txt file into DataFrame
                    df = pd.read_csv(txt_path, sep=",", names=channels)
                    # Add metadata columns
                    df["subject_id"] = subject_id
                    df["study_id"] = study_id
                    df["device_id"] = device_id
                    df["record_name"] = record_name
                    df["device_location"] = device_location
                    df["rows"] = rows
                    # Append to list
                    all_data.append(df)
                else:
                    print(f"Warning: {txt_path} not found.")

# Combine all DataFrames
final_df = pd.concat(all_data, ignore_index=True)

# Save to CSV
final_df.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Data saved to {output_csv}")

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\garin\\OneDrive\\Numpy 2025\\Datathon feb2025\\pads-parkinsons-disease-smartwatch-dataset\\pads-parkinsons-disease-smartwatch-dataset'

In [2]:
# Import necessary libraries
import pandas as pd

df = pd.read_csv(r'C:OneDrive/Numpy 2025/Datathon feb2025/Update PADS files/Patients.csv', encoding='utf-8')

# Function to split the disease_comment column
# Rules:
# 1. All comments start with 'IPS' followed by a space.
# 2. For comments without a comma, everything after 'IPS ' is treated as the disease subtype, and medication and other comments are empty.
# 3. For comments with a comma, we split on the comma: 
#    - Everything after 'IPS ' and before the first comma is disease subtype.
#    - The portion after the comma is other comments. Medication remains empty.

def split_disease_comment(row):
    comment = row['disease_comment']
    if pd.isnull(comment):
        return pd.Series({'disease_subtype': None, 'medication': None, 'other_comments': None})
    # Check for 'IPS'
    if comment.startswith('IPS'):
        # Remove 'IPS' portion, including a possible trailing space
        rest = comment[3:].strip()
        if ',' in rest:
            parts = rest.split(',', 1)
            disease_subtype = parts[0].strip()
            other_comments = parts[1].strip()
            medication = ""
        else:
            disease_subtype = rest
            other_comments = ""
            medication = ""
    else:
        # If doesn't start with IPS, leave empty for extraction
        disease_subtype = ""
        other_comments = ""
        medication = ""
    return pd.Series({'disease_subtype': disease_subtype, 'medication': medication, 'other_comments': other_comments})

# Apply the splitting function
split_columns = df.apply(split_disease_comment, axis=1)

# Append the new columns to the dataframe
df = pd.concat([df, split_columns], axis=1)

# Show the head of resulting dataframe
print('Data cleaning complete. Here is the head of the updated DataFrame:')
print(df.head())

# Additional preliminary analysis: count rows per condition
condition_counts = df['condition'].value_counts()
print('Condition counts:')
print(condition_counts)

# Done
print('done')

FileNotFoundError: [Errno 2] No such file or directory: 'OneDrive/Numpy 2025/Datathon feb2025/Update PADS files/Patients.csv'